# Analysis of 4 Specific Shots for Joe

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import xarray as xr 
import scipy.stats as stat
import sys
import os
import glob
import re
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.magnum.database as ut
import flopter.magnum.utils as mgut
import flopter.core.fitters as fts
import flopter.core.constants as c

In [2]:
# Create analysed dataset metadata 

path_to_datasets = '/home/jleland/data/external/magnum/'
# path_to_datasets = '/home/jleland/data/externy/magnum/'
# path_to_analysed_datasets = 'analysed_2'
# path_to_analysed_datasets = 'analysed_3'
# path_to_analysed_datasets = 'phobos_test'
# path_to_analysed_datasets = 'analysed_4'
# path_to_analysed_datasets = 'test'
path_to_analysed_datasets = 'analysed_3_downsampled'
os.chdir(path_to_datasets)

---
## Saving / loading section
The metadata doesn't need to be reconstructed _every_ time, so we can store it in a .csv file output by pandas. The following 2 methods save and load, use whichever is appropriate

In [3]:
metadata_filename = f'{path_to_analysed_datasets}_xr_metadata.csv'

# Loading the metadata .csv file into a pandas dataframe
try:
    analysed_infos_df = pd.read_csv(metadata_filename).set_index('adc_index')
except:
    # If the .csv file doesn't exist, create it and save it to where we would expect it to be
    analysed_infos_df = mgut.create_analysed_ds_metadata(path_to_analysed_datasets)
    analysed_infos_df.to_csv(metadata_filename)

In [4]:
# Joe's requested shots
indices = [396, 398, 409, 410]

analysed_infos_df.loc[indices]

,filename,shot_number,shot_time,shot_timestamp,sweep_len,time_len
adc_index,,,,,,
396,analysed_3_downsampled/a398_396_66993832985432...,398,2019-06-06 12:48:45,6699383298543284224,1498,50
398,analysed_3_downsampled/a400_398_66993842345516...,400,2019-06-06 12:52:23,6699384234551673856,1497,50
409,analysed_3_downsampled/a411_409_66994013995453...,411,2019-06-06 13:58:59,6699401399545384960,998,50
410,analysed_3_downsampled/a412_410_66994018222159...,412,2019-06-06 14:00:38,6699401822215929856,998,50


## Load adc file metadata

In [5]:
os.chdir('/home/jleland/data/external/magnum/')
# os.chdir('/home/jleland/data/externy/magnum/')
meta_data_ds = xr.open_dataset('all_meta_data.nc')
print(meta_data_ds)

<xarray.Dataset>
Dimensions:                 (shot_number: 523, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) int32 0 1 2 3 4 ... 519 520 521 522
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Data variables:
    adc_filename            (shot_number) object ...
    ts_density              (shot_number, ts_radial_pos) float64 ...
    ts_temperature          (shot_number, ts_radial_pos) float64 ...
    ts_d_density            (shot_number, ts_radial_pos) float64 ...
    ts_d_temperature        (shot_number, ts_radial_pos) float64 ...
    adc_folder              (shot_number) object ...
    adc_calibration_index   (shot_number) object ...
    adc_4_

# Creation of Dataset from adc_file index selection

In [6]:
magnum_probes = lp.MagnumProbes()

def preprocess_sel(ds, sweep=slice(0,997)):
    ds = ds.reset_index('time', drop=True).load()
    return ds.sel(sweep=sweep)

def preprocess_autosel(ds):
    ds = ds.reset_index('time', drop=True).load()
    sweep = slice(*find_sweep_limit(ds))
    return ds.sel(sweep=sweep)

def preprocess_average(ds):
    ds = ds.reset_index('time', drop=True).load()
    sweep_min, sweep_max = find_sweep_limit(ds)

    ds_avg = ds.sel(sweep=slice(sweep_min, sweep_max)).mean('sweep')
    ds_avg = ds_avg.assign({'d_current': ds.std('sweep')['current']})
    return ds_avg

def preprocess_average_downsample(ds, downsample_to=500):
    ds = ds.reset_index('time', drop=True).load()
    dsf = int(len(ds.time) / downsample_to)
    ds = ds.sel(time=slice(0, len(ds.time), dsf))
    ds_avg = ds.mean('sweep')
    ds_avg = ds_avg.assign({'d_current': ds.std('sweep')['current']})
    return ds_avg

In [7]:
analysed_metadata_oi = analysed_infos_df.loc[indices]

files_oi = analysed_metadata_oi['filename'].values
shot_numbers = analysed_metadata_oi['shot_number'].values
shot_numbers_da = xr.DataArray(shot_numbers, dims=['shot_number'], name='shot_number')
min_sweep_len = analysed_metadata_oi['sweep_len'].min()
print(min_sweep_len)

998


In [8]:
meta_data_oi_ds = meta_data_ds.sel(shot_number=shot_numbers)
meta_data_oi_ds.max('ts_radial_pos')

<xarray.Dataset>
Dimensions:                 (shot_number: 4)
Coordinates:
  * shot_number             (shot_number) int32 398 400 411 412
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
Data variables:
    adc_filename            (shot_number) object '2019-06-06 13h 49m 00s TT_06699383298543284340.adc' ... '2019-06-06 15h 00m 48s TT_06699401822215929977.adc'
    ts_density              (shot_number) float64 5.365e+19 ... 7.049e+20
    ts_temperature          (shot_number) float64 4.369 2.538 1.761 1.772
    ts_d_density            (shot_number) float64 0.0 0.0 0.0 0.0
    ts_d_temperature        (shot_number) float64 0.0 0.0 0.0 0.0
    adc_folder              (shot_number) object '2019-06-06_Leland/' ... '2019-06-06_Leland/'
    adc_calibration_index  

In [9]:
tilt_da = meta_data_oi_ds[['shot_tilt', 'adc_4_probe', 'adc_5_probe']]
tilt_da['shot_tilt'] = tilt_da.shot_tilt.round()
tilt_da


<xarray.Dataset>
Dimensions:       (shot_number: 4)
Coordinates:
    ts_timestamp  (shot_number) float64 ...
  * shot_number   (shot_number) int32 398 400 411 412
    adc_time      (shot_number) datetime64[ns] ...
    ts_number     (shot_number) float64 ...
    ts_time       (shot_number) datetime64[ns] ...
    adc_index     (shot_number) float64 ...
Data variables:
    shot_tilt     (shot_number) float64 8.0 8.0 5.0 5.0
    adc_4_probe   (shot_number) object 'S' 'S' 'S' 'S'
    adc_5_probe   (shot_number) object 'L' 'L' 'L' 'L'

In [10]:
# This can't work as the shots are different lengths, something weird going on...

# combined_ds = xr.open_mfdataset(files_oi, concat_dim=shot_numbers_da, preprocess=preprocess_average, combine='nested', parallel=False)
combined_ds = xr.open_mfdataset(files_oi, concat_dim=shot_numbers_da, preprocess=lambda x: preprocess_sel(x, slice(0,693)), combine='nested')
# combined_ds = combined_ds.load()
combined_ds

<xarray.Dataset>
Dimensions:      (direction: 2, probe: 2, shot_number: 4, sweep: 693, time: 50)
Coordinates:
  * probe        (probe) object 'S' 'L'
  * direction    (direction) object 'up' 'down'
  * shot_number  (shot_number) int64 398 400 411 412
Dimensions without coordinates: sweep, time
Data variables:
    voltage      (shot_number, probe, direction, sweep, time) float64 -90.49 ... -53.75
    current      (shot_number, probe, direction, sweep, time) float64 -0.2395 ... -1.224
    shot_time    (shot_number, probe, direction, sweep, time) float64 1.006 ... 7.939
    start_time   (shot_number, probe, direction, sweep) float64 1.006 ... 7.939

In [11]:
combined_all_ds = xr.merge([combined_ds, meta_data_oi_ds.rename({'shot_time':'sweep_time'})])
combined_all_ds

<xarray.Dataset>
Dimensions:                 (direction: 2, probe: 2, shot_number: 4, sweep: 693, time: 50, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'S' 'L'
  * direction               (direction) object 'up' 'down'
  * shot_number             (shot_number) int64 398 400 411 412
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Dimensions without coordinates: sweep, time
Data variables:
    voltage                 (shot_number, probe, direction, sweep, time) float64 -90.49 ... -53.75
    current                 (shot_number, probe, direction, sweep, time) float64 -0.2395 ... -1.224
    shot_time               (shot_number, probe, direction, sw

In [12]:
averaged_ds = combined_all_ds.max('ts_radial_pos').mean(['direction', 'sweep']).set_coords('voltage')
averaged_ds = averaged_ds.assign({'d_current': combined_all_ds.std('sweep', 'direction')['current']})
averaged_ds = averaged_ds.where(averaged_ds.current < 0.16, drop=True)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [13]:
# Plot all averaged IV characteristics

fig, ax = plt.subplots(2, sharex=True, sharey=True)
averaged_ds.sel(probe='S')['current'].plot.line(x='voltage', hue='shot_number', ax=ax[0])
averaged_ds.sel(probe='L')['current'].plot.line(x='voltage', hue='shot_number', ax=ax[1])
ax[0].axhline(0, linestyle='--', color='grey', linewidth=1)
ax[1].axhline(0, linestyle='--', color='grey', linewidth=1)
plt.show()

In [14]:
# Plot individual voltage and current traces for each probe/shot

fig, ax = plt.subplots(2, 2, sharex=True, sharey='row')

sprobe_ds = combined_all_ds.max('ts_radial_pos').mean('direction').sel(probe='S')
lprobe_ds = combined_all_ds.max('ts_radial_pos').mean('direction').sel(probe='L')

sprobe_ds['current'].plot.line(hue='shot_number', ax=ax[0][0])
lprobe_ds['current'].plot.line(hue='shot_number', ax=ax[0][1])
sprobe_ds['voltage'].plot.line(hue='shot_number', ax=ax[1][0])
lprobe_ds['voltage'].plot.line(hue='shot_number', ax=ax[1][1])

plt.show()

ValueError: Line plots are for 1- or 2-dimensional DataArrays. Passed DataArray has 3 dimensions

In [15]:
# Plot individual current & voltage traces for a selection of individual, non-averaged sweeps

for shot_number in combined_all_ds.shot_number.values:
    fig, ax = plt.subplots(2, 2, sharex=True, sharey='row')
    
    sprobe_ds = combined_all_ds.sel(shot_number=shot_number, sweep=slice(None, None, 20)).max('ts_radial_pos').mean('direction').sel(probe='S')
    lprobe_ds = combined_all_ds.sel(shot_number=shot_number, sweep=slice(None, None, 20)).max('ts_radial_pos').mean('direction').sel(probe='L')

#     print(sprobe_ds)
    sprobe_ds['current'].plot.line(hue='sweep', ax=ax[0][0])
    lprobe_ds['current'].plot.line(hue='sweep', ax=ax[0][1])
    sprobe_ds['voltage'].plot.line(hue='sweep', ax=ax[1][0])
    lprobe_ds['voltage'].plot.line(hue='sweep', ax=ax[1][1])

plt.show()

In [23]:
# copied from concatted_ds_analysis_0.ipynb
mandatory_labels = [
    'shot',
    'probe', 
    'B',
    'ts_temp',
    'ts_dens',
    'shot_time',
    'fit_success_fl',
]
fit_param_labels = [
    'temp',
    'd_temp',
    'isat', 
    'd_isat',
    'a',
    'd_a',
    'v_f',
    'd_v_f', 
    'dens', 
    'd_dens'
]
all_labels = mandatory_labels + fit_param_labels
fit_df = pd.DataFrame(columns=all_labels)

probes = ['S', 'L']

for shot in averaged_ds.shot_number.values:
    shot_ds = averaged_ds.sel(shot_number=shot)
    for probe in probes:
        probe_shot_ds = shot_ds.sel(probe=probe)
        probe_shot_ds = probe_shot_ds.where(np.isfinite(probe_shot_ds['voltage']), drop=True).where(np.isfinite(probe_shot_ds['current']), drop=True)
        
        alpha = np.radians(probe_shot_ds['shot_tilt'].values[0])
        
        if len(probe_shot_ds.time) == 0:
            print('Time has no length, continuing...')
            continue
#         probe_shot_ds = probe_shot_ds
        
#         print(probe_shot_ds)
        shot_iv = iv.IVData(probe_shot_ds['voltage'].values,
                            -probe_shot_ds['current'].values,
                            probe_shot_ds['shot_time'].values,
                            sigma=probe_shot_ds['d_current'])
        
        fig, ax = plt.subplots()
        ax.errorbar(shot_iv['V'], shot_iv['I'], yerr=shot_iv['sigma'], ecolor='silver', color='silver', marker='+')
        
        fit_params = {}
        fitter = fts.FullIVFitter()
        try:
#             shot_fit = shot_iv.multi_fit(sat_region=-40)
            shot_fit = fitter.fit_iv_data(shot_iv, sigma=shot_iv['sigma'])
            fit_params = {
                'fit_success_fl': True,
                'temp': shot_fit.get_temp(),
                'd_temp': shot_fit.get_temp_err(),
                'isat': shot_fit.get_isat(), 
                'd_isat': shot_fit.get_isat_err(),
                'a': shot_fit.get_sheath_exp(), 
                'd_a': shot_fit.get_sheath_exp_err(),
                'v_f': shot_fit.get_floating_pot(),
                'd_v_f': shot_fit.get_floating_pot_err(),
                'dens': magnum_probes[probe].get_density(shot_fit.get_isat(), shot_fit.get_temp(), alpha=alpha)[0],
                'd_dens': magnum_probes[probe].get_d_density(
                    shot_fit.get_isat(), 
                    shot_fit.get_isat_err(), 
                    shot_fit.get_temp(), 
                    shot_fit.get_temp_err(),
                    alpha=alpha
                )[0]
            
            }
            ax.plot(*shot_fit.get_fit_plottables())
        except RuntimeError as e:
            print(f'WARNING: Failed on shot {shot} with probe {probe}')
            fit_params = {label: np.NaN for label in fit_param_labels}
            fit_params['fit_success_fl'] = False
            
        
        fit_df = fit_df.append({
            'shot': shot,
            'probe': probe, 
            'B': np.around(probe_shot_ds['shot_b_field'].mean().values, decimals=1),
            'ts_temp': probe_shot_ds['ts_temperature'].mean().values,
            'ts_dens': probe_shot_ds['ts_density'].mean().values,
            'shot_time': probe_shot_ds['ts_time'].values,
            **fit_params
        }, ignore_index=True)
        

In [24]:
fit_df.to_csv('joe_fit_data.csv')
fit_df

,shot,probe,B,ts_temp,ts_dens,shot_time,fit_success_fl,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f,dens,d_dens
0,398,S,1.5,4.369459447424046,5.364572207500493e+19,2019-06-06 12:48:45.483818,True,3.225564,0.083649,0.181375,0.002203,0.046504,0.001885,-22.403745,0.034886,1.986804e+19,1.644698e+18
1,398,L,1.5,4.369459447424046,5.364572207500493e+19,2019-06-06 12:48:45.483818,True,2.961695,0.085059,0.081597,0.001433,0.010025,0.002377,-18.290009,0.057125,5.834852e+18,3.829714e+17
2,400,S,1.5,2.5384518638027314,2.6261608290229787e+20,2019-06-06 12:52:23.415253,True,2.007685,0.013321,0.564493,0.001026,0.015133,0.000185,-14.817831,0.004674,7.837754e+19,6.343928e+18
3,400,L,1.5,2.5384518638027314,2.6261608290229787e+20,2019-06-06 12:52:23.415253,True,3.600643,0.157970,0.509277,0.007825,0.006132,0.002246,-13.443836,0.055867,3.302874e+19,2.218236e+18
4,411,S,1.5,1.7605041688918084,8.437227204338191e+20,2019-06-06 13:58:59.951787,True,0.421113,0.109696,1.267321,0.037512,0.001552,0.000953,-7.618189,0.050570,5.457774e+20,9.558652e+19
5,411,L,1.5,1.7605041688918084,8.437227204338191e+20,2019-06-06 13:58:59.951787,True,0.707659,0.056167,2.135215,0.045434,0.002096,0.001054,-7.350518,0.021763,4.044982e+20,3.713076e+19
6,412,S,1.5,1.7720984299581886,7.049155171000286e+20,2019-06-06 14:00:38.362442,True,0.799687,0.060525,1.020039,0.013958,0.004243,0.000708,-9.035841,0.018241,3.187757e+20,3.832328e+19
7,412,L,1.5,1.7720984299581886,7.049155171000286e+20,2019-06-06 14:00:38.362442,True,1.423982,0.059813,1.720236,0.019305,0.002938,0.000894,-8.567936,0.023394,2.297327e+20,1.917355e+19


In [19]:
fig, ax = plt.subplots()
ax.plot('ts_temp', 'ts_dens', data=fit_df)
ax.errorbar('temp', 'dens', xerr='d_temp', yerr='d_dens', data=fit_df)

<ErrorbarContainer object of 3 artists>

In [126]:
probe_s = fit_df.loc[fit_df['probe'] == 'S'].drop(columns='probe').set_index('shot')
probe_l = fit_df.loc[fit_df['probe'] == 'L'].drop(columns='probe').set_index('shot')

In [127]:
probe_da = xr.DataArray(['S', 'L'], dims=['probe'], name='probe')
fit_ds = xr.concat([probe_s.to_xarray(), probe_l.to_xarray()], probe_da).rename(shot='shot_number')
fit_ds

<xarray.Dataset>
Dimensions:         (probe: 2, shot_number: 3)
Coordinates:
  * shot_number     (shot_number) int64 398 400 411
  * probe           (probe) <U1 'S' 'L'
Data variables:
    B               (probe, shot_number) float64 1.5 1.5 1.5 1.5 1.5 1.5
    ts_temp         (probe, shot_number) object 4.369459447424046 ... 1.7605041688918084
    ts_dens         (probe, shot_number) object 5.364572207500493e+19 ... 8.437227204338191e+20
    fit_success_fl  (probe, shot_number) object True True True True True True
    temp            (probe, shot_number) float64 3.226 2.008 ... 3.601 0.7077
    d_temp          (probe, shot_number) float64 0.08365 0.01332 ... 0.05617
    isat            (probe, shot_number) float64 0.1814 0.5645 ... 0.5093 2.135
    d_isat          (probe, shot_number) float64 0.002203 0.001026 ... 0.04543
    a               (probe, shot_number) float64 0.0465 0.01513 ... 0.002096
    d_a             (probe, shot_number) float64 0.001885 0.0001848 ... 0.001054
    v_f

In [147]:
fig, ax = plt.subplots()

colours = ['red', 'blue', 'orange']

for i, shot in enumerate(fit_ds.shot_number.values):
    ax.plot('ts_temp', 'ts_dens', '*', data=fit_df.loc[fit_df['shot'] == shot], label=f'{shot} - TS', color=colours[i])
    
    ds = fit_ds.sel(probe='S', shot_number=shot)
    ax.errorbar(ds['temp'].values, ds['dens'].values, xerr=ds['d_temp'].values, yerr=ds['d_dens'].values, label=f'{shot} - S', fmt='o', color=colours[i])
    
    ds = fit_ds.sel(probe='L', shot_number=shot)
    ax.errorbar(ds['temp'].values, ds['dens'].values, xerr=ds['d_temp'].values, yerr=ds['d_dens'].values, label=f'{shot} - L', fmt='s', color=colours[i])
    ax.legend()

# Analysis of individual shots first

In [21]:
ds_list = {shot_numbers[i]: xr.open_dataset(fn) for i, fn in enumerate(files_oi)}

In [23]:
ds_list

{398: <xarray.Dataset>
 Dimensions:     (direction: 2, probe: 2, sweep: 1499, time: 5000)
 Coordinates:
   * time        (time) float64 0.0 1e-06 2e-06 ... 0.004997 0.004998 0.004999
   * direction   (direction) object 'up' 'down'
   * probe       (probe) object 'S' 'L'
 Dimensions without coordinates: sweep
 Data variables:
     voltage     (probe, direction, sweep, time) float64 ...
     current     (probe, direction, sweep, time) float64 ...
     shot_time   (probe, direction, sweep, time) float64 ...
     start_time  (probe, direction, sweep) float64 ..., 400: <xarray.Dataset>
 Dimensions:     (direction: 2, probe: 2, sweep: 509, time: 5000)
 Coordinates:
   * time        (time) float64 0.0 1e-06 2e-06 ... 0.004997 0.004998 0.004999
   * direction   (direction) object 'up' 'down'
   * probe       (probe) object 'S' 'L'
 Dimensions without coordinates: sweep
 Data variables:
     voltage     (probe, direction, sweep, time) float64 ...
     current     (probe, direction, sweep, time)

In [18]:
for ds in ds_list:
    fig, ax = plt.subplots()
    ds.mean('direction').max('time')['current'].plot.line(hue='probe', x='sweep', ax=ax)

## Raw Data Time!

Shots 400 and 411 seem to be only looking at the end after the beam dump has gone down. Time to look at the raw data!

In [21]:
select_file_info = analysed_infos_df.loc[indices]
select_file_info

,filename,shot_number,shot_time,shot_timestamp,sweep_len,time_len
adc_index,,,,,,
396,analysed_3_downsampled/a398_396_66993832985432...,398,2019-06-06 12:48:45,6699383298543284224,1498,50
398,analysed_3_downsampled/a400_398_66993842345516...,400,2019-06-06 12:52:23,6699384234551673856,1497,50
409,analysed_3_downsampled/a411_409_66994013995453...,411,2019-06-06 13:58:59,6699401399545384960,998,50


In [11]:
print(path_to_datasets)

/home/jleland/data/external/magnum/


In [12]:
for i, row in select_file_info.iterrows():
    print(i, )
    shot_date = row['shot_time'].split()[0]
    shot_timestamp = str(row['shot_timestamp'])[:-4]
    print(shot_date, shot_timestamp)
    glob_str = f'{shot_date}_Leland/*{shot_timestamp}*.adc'
    print(glob_str)
    print(glob.glob(glob_str))

396
2019-06-06 669938329854328
2019-06-06_Leland/*669938329854328*.adc
['2019-06-06_Leland/2019-06-06 13h 49m 00s TT_06699383298543284340.adc']
398
2019-06-06 669938423455167
2019-06-06_Leland/*669938423455167*.adc
['2019-06-06_Leland/2019-06-06 13h 52m 38s TT_06699384234551673455.adc']
409
2019-06-06 669940139954538
2019-06-06_Leland/*669940139954538*.adc
['2019-06-06_Leland/2019-06-06 14h 59m 10s TT_06699401399545384568.adc']


In [1]:
import analysis.scripts.magnum.magnum_postrun_initial_analysis as mpia

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


In [2]:
mpia.aia_mapping_wrapper(398)


 Analysing shot 398...
Attempting analysis on shot 398
"2019-06-06_Leland/" 		 "2019-06-06 13h 49m 00s TT_06699383298543284340.adc"
Creating magopter object
Running an offline magopter object, operating in offline mode.

 ...Finished shot 398


Traceback (most recent call last):
  File "/home/jleland/coding/projects/flopter/analysis/scripts/magnum/magnum_postrun_initial_analysis.py", line 186, in aia_mapping_wrapper
    downsampling_factor=downsampling_factor)
  File "/home/jleland/coding/projects/flopter/analysis/scripts/magnum/magnum_postrun_initial_analysis.py", line 44, in averaged_iv_analysis
    magopter.prepare(down_sampling_rate=dsr, roi_b_plasma=True, filter_arcs_fl=False, crit_freq=None, crit_ampl=None)
  File "/home/jleland/coding/projects/flopter/flopter/magnum/magoptoffline.py", line 151, in prepare
    top = sig.argrelmax(smoothed_voltage, order=100)[0]
  File "/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/signal/_peak_finding.py", line 194, in argrelmax
    return argrelextrema(data, np.greater, axis, order, mode)
  File "/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/signal/_peak_finding.py", line 249, in argrelextrema
    axis, order, mode)
  File

In [3]:
shot_numbers = [398, 400, 411]
magopters = []
dses = []
for shot_number in shot_numbers:
    folder, adc_file, output_tag, ts_temp, ts_dens, probe_designations, shunt_resistance, downsampling_factor, \
    cabling_resistance, sweep_range = mpia.get_shot_info_for_analysis(shot_number)

    magopter, ds_full = mpia.averaged_iv_analysis(folder, adc_file, output_tag, probe_designations=probe_designations,
                                                  shunt_resistance=shunt_resistance, cabling_resistance=cabling_resistance,
                                                  downsampling_factor=100, dealloc=False)
    magopters.append(magopter)
    dses.append(ds_full)

"2019-06-06_Leland/" 		 "2019-06-06 13h 49m 00s TT_06699383298543284340.adc"
Creating magopter object
Running an offline magopter object, operating in offline mode.
0: 2997, 1: 2997
"2019-06-06_Leland/" 		 "2019-06-06 13h 52m 38s TT_06699384234551673455.adc"
Creating magopter object
Running an offline magopter object, operating in offline mode.
0: 2995, 1: 2995
"2019-06-06_Leland/" 		 "2019-06-06 14h 59m 10s TT_06699401399545384568.adc"
Creating magopter object
Running an offline magopter object, operating in offline mode.
0: 1996, 1: 1996


In [21]:
max_current = ds_full['current'].mean('direction').max('time')

plt.figure()
max_current.plot.line(x='sweep', hue='probe')
np.abs(max_current - max_current.median('sweep')).plot.line(x='sweep', hue='probe')

In [20]:
fig, ax = plt.subplots(3)
for i, ds in enumerate(dses):
    max_current = ds['current'].mean('direction').max('time')
    max_current_trim = max_current.where(np.abs(max_current - max_current.median('sweep')) < 2.5 * max_current.std('sweep'))
    max_current.plot.line(x='sweep', hue='probe', ax=ax[i])
    max_current_trim.plot.line(x='sweep', hue='probe', ax=ax[i])
    for j in range(2):
        median = max_current.isel(probe=j).median('sweep')
        ax[i].axhline(y=max_current.median('sweep'), **c.AX_LINE_DEFAULTS, label='j')

Discovery: the correct sweep range is selected when downsampling, but not when looking at the whole file, might be some bug with arrays being too large?

In [38]:
fig, ax = plt.subplots(3)
for i, ds in enumerate(dses):
    max_current = ds['current'].mean('direction').max('time').isel(probe=1)
    print(len(max_current))
    
    median = max_current.median('sweep')
    std = max_current.std('sweep')
    ax[i].axhline(y=median, color='grey', linewidth=.9, linestyle='--', label='median')
    ax[i].axhline(y=median + 1.5 * std, color='gray', linewidth=.9, linestyle=':', label='std')
    ax[i].axhline(y=median - 1.5 * std, color='gray', linewidth=.9, linestyle=':',)
    
    max_current_trim = max_current.where(np.abs(max_current - max_current.median()) < 1.5 * max_current.std(), drop=True)
    print(len(max_current_trim))
    
    median = max_current_trim.median('sweep')
    std = max_current_trim.std('sweep')
    ax[i].axhline(y=median, color='red', linewidth=.9, linestyle='--', label='median_trim')
    ax[i].axhline(y=median + std, color='red', linewidth=.9, linestyle=':', label='std_trim')
    ax[i].axhline(y=median - std, color='red', linewidth=.9, linestyle=':',)

    max_current_trim_2 = max_current_trim.where(np.abs(max_current_trim - max_current_trim.median('sweep')) < 2.5 * max_current_trim.std('sweep'), drop=True)
    print(len(max_current_trim_2))
    
    max_current.plot.line(x='sweep', ax=ax[i])
    max_current_trim.plot.line(x='sweep', ax=ax[i])
    max_current_trim_2.plot.line(x='sweep', ax=ax[i])
    
    print(f'sweep_limits found: ({max_current_trim_2["sweep"].min().values} , {max_current_trim_2["sweep"].max().values})')
    
    ax[i].legend()

1498
1388
1353
sweep_limits found: (0 , 1352)
1497
1000
986
sweep_limits found: (0 , 985)
998
701
694
sweep_limits found: (0 , 693)


In [59]:
def find_sweep_limit(ds, probe=0):
    max_current = ds['current'].mean('direction').max('time').isel(probe=1)
    
    max_current_trim = max_current.where(exclude_outliers_cond(max_current, 1.5), drop=True)
    max_current_trim_2 = max_current_trim.where(exclude_outliers_cond(max_current_trim, 2.5), drop=True)
    
    return max_current_trim_2["sweep"].min().values, max_current_trim_2["sweep"].max().values

def exclude_outliers_cond(da, n=2):
    return np.abs(da - da.median()) < n * da.std()